In [ ]:
from llama_index import GPTVectorStoreIndex, download_loader
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory

GoogleDocsReader = download_loader('GoogleDocsReader')

gdoc_ids = ['1wf-y2pd9C878Oh-FmLH7Q_BQkljdm6TQal-c1pUfrec']
loader = GoogleDocsReader()
documents = loader.load_data(document_ids=gdoc_ids)
index = GPTVectorStoreIndex(documents)

tools = [
    Tool(
        name="Google Doc Index",
        func=lambda q: index.query(q),
        description=f"Useful when you want answer questions about the Google Documents.",
    ),
]
llm = OpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(
    tools, llm, agent="zero-shot-react-description", memory=memory
)

output = agent_chain.run(input="Where did the author go to school?")

In [5]:
%reload_ext autoreload
%autoreload 2

from setlist_app import SetListApp

app = SetListApp("tmG3-KHsciHD1mS5y58b1FIv5NMPccWTKN8E")
r = await app.get_setList(artistName="Taylor Swift")
print(f"run: {await r.json()}")


run: {
   "type" : "setlists",
   "itemsPerPage" : 20,
   "page" : 1,
   "total" : 965,
   "setlist" : [ {
      "id" : "13a6b159",
      "versionId" : "g6392e6e3",
      "eventDate" : "04-06-2023",
      "lastUpdated" : "2023-06-03T15:15:09.660+0000",
      "artist" : {
         "mbid" : "20244d07-534f-4eff-b4d4-930878889970",
         "name" : "Taylor Swift",
         "sortName" : "Swift, Taylor",
         "disambiguation" : "",
         "url" : "https://www.setlist.fm/setlists/taylor-swift-3bd6bc5c.html"
      },
      "venue" : {
         "id" : "4bd637ce",
         "name" : "Soldier Field",
         "city" : {
            "id" : "4887398",
            "name" : "Chicago",
            "state" : "Illinois",
            "stateCode" : "IL",
            "coords" : {
               "lat" : 41.850033,
               "long" : -87.6500523
            },
            "country" : {
               "code" : "US",
               "name" : "United States"
            }
         },
         "url" : 